In [1]:
import os
import sys
from pathlib import Path

import cv2
import numpy as np

def setup_project_path() -> Path:
    current = Path.cwd()
    while current != current.parent and not (current / "craft").exists():
        current = current.parent
    if not (current / "craft").exists():
        raise RuntimeError("Could not find project_root containing 'craft' directory.")
    return current


project_root = setup_project_path()
sys.path.insert(0, str(project_root))


In [3]:
def imwrite_unicode(path, img) -> bool:
    path = str(path)
    ext = os.path.splitext(path)[1]
    ok, buf = cv2.imencode(ext, img)
    if not ok:
        return False
    with open(path, "wb") as f:
        f.write(buf.tobytes())
    return True


def imread_unicode(path, flags=cv2.IMREAD_COLOR):
    path = str(path)
    try:
        with open(path, "rb") as f:
            data = f.read()
        img_array = np.frombuffer(data, np.uint8)
        img = cv2.imdecode(img_array, flags)
        return img
    except Exception as e:
        print("[imread_unicode ERROR]", e)
        return None


In [5]:
def remove_noise_by_craft_score_far_only(
    ink_white_on_black: np.ndarray,     # uint8, 0/255, white=ink
    score_text: np.ndarray,             # float map (CRAFT score_text.npy)
    *,
    text_prob_thresh: float = 0.25,     # 글씨 영역으로 인정할 score 임계값
    low_prob_thresh: float = 0.02,      # "확률이 거의 없다" 기준 (컴포넌트 내부 max score)
    dist_thresh_px: float = 40.0,       # 글씨에서 이 거리 이상 떨어진 잉크만 제거 대상
    protect_dilate_r: int = 8,          # 글씨 영역 주변 보호(팽창) 반경
    connectivity: int = 8,
) -> np.ndarray:
    """
    - CRAFT text score가 거의 0이고(low_prob_thresh 이하),
    - CRAFT가 인식한 글씨 영역(text_prob_thresh 이상)으로부터 충분히 멀리(dist_thresh_px 이상)
    떨어진 잉크 컴포넌트만 제거.
    """
    if ink_white_on_black.ndim != 2:
        raise ValueError("ink_white_on_black must be a single-channel grayscale/binary image.")
    ink = (ink_white_on_black > 0).astype(np.uint8) * 255
    H, W = ink.shape[:2]

    # score_text shape normalize
    s = score_text
    if s.ndim == 3:
        s = s.squeeze()
    if s.ndim != 2:
        raise ValueError("score_text must be 2D (H,W) or squeezable to 2D.")
    s = s.astype(np.float32)

    # Resize score map to ink size if needed
    if s.shape[0] != H or s.shape[1] != W:
        s = cv2.resize(s, (W, H), interpolation=cv2.INTER_LINEAR)

    # Build "text region" from CRAFT score
    text_region = (s >= float(text_prob_thresh)).astype(np.uint8)  # 0/1

    # Protect 주변 글씨가 잘리지 않도록 dilate
    if protect_dilate_r > 0:
        k = cv2.getStructuringElement(
            cv2.MORPH_ELLIPSE, (2 * protect_dilate_r + 1, 2 * protect_dilate_r + 1)
        )
        text_region = cv2.dilate(text_region, k, iterations=1)

    # Distance transform to nearest text pixel
    # distTransform input expects non-zero as free-space, 0 as obstacle.
    inv = (1 - text_region).astype(np.uint8) * 255
    dist = cv2.distanceTransform(inv, distanceType=cv2.DIST_L2, maskSize=3)

    # Connected components on ink
    num, labels, stats, _ = cv2.connectedComponentsWithStats(ink, connectivity=connectivity)

    out = ink.copy()
    for i in range(1, num):
        comp = (labels == i)

        # 1) 글씨로부터 충분히 멀리 떨어진 컴포넌트만 고려
        comp_min_dist = float(dist[comp].min())
        if comp_min_dist < float(dist_thresh_px):
            continue

        # 2) CRAFT score가 거의 0인 컴포넌트만 제거
        comp_max_score = float(s[comp].max())
        if comp_max_score <= float(low_prob_thresh):
            out[comp] = 0

    return out


In [7]:
def run_score_denoise_single(
    img_path: Path,
    score_path: Path,
    out_path: Path,
    *,
    text_prob_thresh=0.25,
    low_prob_thresh=0.02,
    dist_thresh_px=40.0,
    protect_dilate_r=8,
    connectivity=8,
):
    img = imread_unicode(str(img_path), flags=cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise RuntimeError(f"Failed to read image: {img_path}")

    # Ensure binary: white ink on black
    _, ink = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY)

    score = np.load(str(score_path))

    cleaned = remove_noise_by_craft_score_far_only(
        ink_white_on_black=ink,
        score_text=score,
        text_prob_thresh=text_prob_thresh,
        low_prob_thresh=low_prob_thresh,
        dist_thresh_px=dist_thresh_px,
        protect_dilate_r=protect_dilate_r,
        connectivity=connectivity,
    )

    out_path.parent.mkdir(parents=True, exist_ok=True)
    ok = imwrite_unicode(str(out_path), cleaned)
    if not ok:
        raise RuntimeError(f"Failed to write image: {out_path}")

    return cleaned


In [9]:
def find_score_file(score_dir: Path, stem: str) -> Path | None:
    """
    기본 가정:
      - res_{stem}_score_text.npy
      - {stem}_score_text.npy
    둘 중 존재하는 것을 반환
    """
    cand1 = score_dir / f"res_{stem}_score_text.npy"
    if cand1.exists():
        return cand1
    cand2 = score_dir / f"{stem}_score_text.npy"
    if cand2.exists():
        return cand2
    return None


In [11]:
def run_score_denoise_folder(
    in_dir: Path,
    score_dir: Path,
    out_dir: Path,
    *,
    exts=(".png", ".jpg", ".jpeg", ".bmp"),
    text_prob_thresh=0.25,
    low_prob_thresh=0.02,
    dist_thresh_px=40.0,
    protect_dilate_r=8,
    connectivity=8,
    verbose=True,
):
    in_dir = Path(in_dir)
    score_dir = Path(score_dir)
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    files = sorted([p for p in in_dir.iterdir() if p.is_file() and p.suffix.lower() in exts])

    if verbose:
        print(f"[IN   ] {in_dir}")
        print(f"[SCORE] {score_dir}")
        print(f"[OUT  ] {out_dir}")
        print(f"[INFO ] {len(files)} images")

    processed = 0
    skipped = 0

    for idx, img_path in enumerate(files, 1):
        stem = img_path.stem
        score_path = find_score_file(score_dir, stem)

        if score_path is None:
            skipped += 1
            if verbose:
                print(f"[SKIP] score missing: {img_path.name}")
            continue

        out_path = out_dir / img_path.name

        try:
            run_score_denoise_single(
                img_path=img_path,
                score_path=score_path,
                out_path=out_path,
                text_prob_thresh=text_prob_thresh,
                low_prob_thresh=low_prob_thresh,
                dist_thresh_px=dist_thresh_px,
                protect_dilate_r=protect_dilate_r,
                connectivity=connectivity,
            )
            processed += 1
        except Exception as e:
            print(f"[ERR ] {img_path.name}: {e}")
            continue

        if verbose and (idx % 10 == 0 or idx == len(files)):
            print(f"  [{idx}/{len(files)}] processed={processed}, skipped={skipped}")

    if verbose:
        print(f"[DONE] processed={processed}, skipped={skipped}")


In [17]:
# ============================
# 사용자 지정 경로 (여기만 수정)
# ============================

# 1차 정규화(전처리) 이미지 폴더
IN_DIR = project_root / "craft" / "images_normalized"

# score_text.npy 폴더
SCORE_DIR = project_root / "craft" / "results" / "score_text"

# 출력 폴더 (2차 정규화 결과)
OUT_DIR = project_root / "craft" / "images_normalized2"


# ============================
# 실행
# ============================
run_score_denoise_folder(
    in_dir=IN_DIR,
    score_dir=SCORE_DIR,
    out_dir=OUT_DIR,
    # ---- 튜닝 파라미터 ----
    text_prob_thresh=0.25,   # 글씨로 인정할 score
    low_prob_thresh=0.02,    # "확률이 거의 없다" 기준
    dist_thresh_px=40.0,     # 글씨에서 충분히 멀리 떨어진 점만 제거
    protect_dilate_r=8,      # 글씨 주변 보호 반경
    connectivity=8,
    verbose=True
)


[IN   ] D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\craft\images_normalized
[SCORE] D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\craft\results\score_text
[OUT  ] D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\craft\images_normalized2
[INFO ] 8 images
  [8/8] processed=8, skipped=0
[DONE] processed=8, skipped=0
